In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras.applications import mobilenet #, vgg16, inception_v3, resnet50, 
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

C:\Users\leetw\Anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### prepare data

In [2]:
data_dir = 'E:/Datasets/PathoBarIlan/Case8'

In [3]:
def load_data_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    imgs = []
    labels = []
    for row in tqdm(df.iterrows()):
        row = row[1]
#         print(row)
        file_path = row.loc["filepath"]
        if file_path.endswith('.png'):
            imgs.append(plt.imread(file_path))
        else:
            imgs.append(np.load(file_path))
        labels.append(row.loc["label"])
    return np.array(imgs), to_categorical(np.array(labels))

In [4]:
x_train, y_train = load_data_from_csv(data_dir+"/train.csv")
x_test, y_test = load_data_from_csv(data_dir+"/test.csv")
x_eval, y_eval = load_data_from_csv(data_dir+"/eval.csv")

130it [00:25,  5.13it/s]
40it [00:07,  5.01it/s]
21it [00:04,  4.97it/s]


In [5]:
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))
print(len(x_eval), len(y_eval))

130 130
40 40
21 21


#### build/train model

In [6]:
mobilenet_model = mobilenet.MobileNet(include_top=True, weights=None, input_shape=x_train[0].shape, classes=2, dropout=0.2)

C:\Users\leetw\Anaconda3\envs\keras\lib\site-packages\keras\applications\imagenet_utils.py:258: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 40 input channels.
  str(input_shape[-1]) + ' input channels.')


In [7]:
mobilenet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 40)      0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 202, 202, 40)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 100, 100, 32)      11520     
_________________________________________________________________
conv1_bn (BatchNormalization (None, 100, 100, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 100, 100, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 102, 102, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 100, 100, 32)      288       
__________

_________________________________________________________________
conv_pad_9 (ZeroPadding2D)   (None, 15, 15, 512)       0         
_________________________________________________________________
conv_dw_9 (DepthwiseConv2D)  (None, 13, 13, 512)       4608      
_________________________________________________________________
conv_dw_9_bn (BatchNormaliza (None, 13, 13, 512)       2048      
_________________________________________________________________
conv_dw_9_relu (Activation)  (None, 13, 13, 512)       0         
_________________________________________________________________
conv_pw_9 (Conv2D)           (None, 13, 13, 512)       262144    
_________________________________________________________________
conv_pw_9_bn (BatchNormaliza (None, 13, 13, 512)       2048      
_________________________________________________________________
conv_pw_9_relu (Activation)  (None, 13, 13, 512)       0         
_________________________________________________________________
conv_pad_1

In [8]:
optimizer = Adam(lr=1e-3)
mobilenet_model.compile(loss="binary_crossentropy", optimizer=optimizer)
lrReduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, verbose=1, min_lr=1e-6)
chkpnt = ModelCheckpoint("my_models/model_spec", save_best_only=True)

In [9]:
mobilenet_model.fit(x=x_train, y=y_train, epochs=1000, validation_data=(x_eval, y_eval), batch_size=1, verbose=2, callbacks=[chkpnt, lrReduce], shuffle=True)

Train on 130 samples, validate on 21 samples
Epoch 1/1000
 - 16s - loss: 0.7549 - val_loss: 0.9328
Epoch 2/1000
 - 7s - loss: 0.6801 - val_loss: 2.0674
Epoch 3/1000
 - 7s - loss: 0.7027 - val_loss: 2.1306
Epoch 4/1000
 - 8s - loss: 0.6770 - val_loss: 0.8053
Epoch 5/1000
 - 8s - loss: 0.6153 - val_loss: 1.0615
Epoch 6/1000
 - 7s - loss: 0.5410 - val_loss: 1.1390
Epoch 7/1000
 - 7s - loss: 0.4857 - val_loss: 1.8982
Epoch 8/1000
 - 7s - loss: 0.6174 - val_loss: 0.7832
Epoch 9/1000
 - 7s - loss: 0.0659 - val_loss: 0.3700
Epoch 10/1000
 - 7s - loss: 0.0140 - val_loss: 0.2117
Epoch 11/1000
 - 7s - loss: 0.0078 - val_loss: 0.1618
Epoch 12/1000
 - 7s - loss: 0.0052 - val_loss: 0.1428
Epoch 13/1000
 - 7s - loss: 0.0039 - val_loss: 0.1260
Epoch 14/1000
 - 7s - loss: 0.0029 - val_loss: 0.1194
Epoch 15/1000
 - 7s - loss: 0.0022 - val_loss: 0.1024
Epoch 16/1000
 - 7s - loss: 0.0019 - val_loss: 0.0960
Epoch 17/1000
 - 7s - loss: 0.0017 - val_loss: 0.0919
Epoch 18/1000
 - 7s - loss: 0.0015 - val_loss

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\leetw\Anaconda3\envs\keras\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-df47ce450169>", line 1, in <module>
    mobilenet_model.fit(x=x_train, y=y_train, epochs=1000, validation_data=(x_eval, y_eval), batch_size=1, verbose=2, callbacks=[chkpnt, lrReduce], shuffle=True)
  File "C:\Users\leetw\Anaconda3\envs\keras\lib\site-packages\keras\engine\training.py", line 1705, in fit
    validation_steps=validation_steps)
  File "C:\Users\leetw\Anaconda3\envs\keras\lib\site-packages\keras\engine\training.py", line 1221, in _fit_loop
    ins_batch = _slice_arrays(ins[:-1], batch_ids) + [ins[-1]]
  File "C:\Users\leetw\Anaconda3\envs\keras\lib\site-packages\keras\engine\training.py", line 385, in _slice_arrays
    return [None if x is None else x[start] for x in arrays]
  File "C:\Users\leetw\Anaconda3\envs\keras\lib\site-packages\keras\e

KeyboardInterrupt: 

#### evaluate

In [10]:
y_pred = mobilenet_model.predict(x_test)
y_pred_bool = y_pred.argmax(axis=1)
y_pred_bool

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [11]:
y_test_bool = y_test.argmax(axis=1)
y_test_bool

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [12]:
(y_test_bool != y_pred_bool).mean()

0.0

In [18]:
from keras.utils.vis_utils import plot_model
plot_model(mobilenet_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [17]:
! pip install pydot

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
